In [4]:
import json
from sklearn.externals import joblib
import os
import re
import numpy as np

In [10]:
docsUrlsPath = '../data/urls.docs.txt'
textdataPath = '../data/textdata'
templateJson = '{:d}.json'
queriesDocsPath = '../data/queries.docs.txt'
queriesPath = '../data/queries.numerate.txt'

pat = re.compile(r'\d+')
procNumbs = set(map(lambda x: int(pat.search(x).group(0)), os.listdir(textdataPath)))

queriesDict = {}
with open(queriesPath) as inputFile:
    for line in inputFile:
        number, query = line.strip().split('\t')
        queriesDict[number] = query
        
queriesDocsDict = json.load(open(queriesDocsPath))

docsUrlsDict = {}
with open(docsUrlsPath) as inputFile:
    for line in inputFile:
        number, url, path = line.strip().split('\t')
        docsUrlsDict[number] = (url, path)

In [3]:
class TfIdfBank:
    tfIdfNumberPagesPath = '../data/tfIdfNumberPages'
    tfIdfFeaturesPath = '../data/tfidfFeatutes'
    
    def __init__(self, tfIdfMatrix):
        numberPages = json.load(open(TfIdfBank.tfIdfNumberPagesPath))
        features = json.load(open(TfIdfBank.tfIdfFeaturesPath))
        
        self._indexPages = np.full(27000, -1, dtype=np.int)
        self._indexPages[numberPages] = range(len(numberPages))
        self._tfIdfMatrix = tfIdfMatrix#joblib.load(TfIdfFeatures.tfIdfPagesPath)        
        self._indexFeatures = dict(map(lambda x: (x[1], x[0]), enumerate(features)))
        
    def tfidf(self, word, numberPage):
        indexPage = self._indexPages[numberPage]
        if indexPage == -1:
            return None
        indexFeature = self._indexFeatures.get(word.strip().lower(), -1)
        if indexFeature == -1:
            return 0
        return self._tfIdfMatrix[indexPage, indexFeature]    

In [12]:
bold  = []
inclined = []
header = []
for numb in procNumbs:
    filename = templateJson.format(numb)
    path = '{:s}/{:s}'.format(textdataPath, filename)
    pageDict = json.load(open(path))
    tests = pageDict["text"].get('b', [''[])
    bold.append(pageDict["text"].get('b', ''))
    keywords.append(pageDict["keywords"])
    descriptions.append(pageDict["description"])

In [14]:
json.dump(titles, open('../data/subdata/titles', 'w'))
json.dump(keywords, open('../data/subdata/keywords', 'w'))
json.dump(descriptions, open('../data/subdata/description', 'w'))
json.dump(procNumbs, open('../data/subdata/procNumbs', 'w'))

In [4]:
titles = json.load(open('../data/subdata/titles'))
keywords = json.load(open('../data/subdata/keywords'))
descriptions = json.load(open('../data/subdata/description'))

In [12]:
tfIdfVectorizerPath = '../data/models/tfIdfVectorizer.pkl'
tfIdfVect = joblib.load(tfIdfVectorizerPath)

In [13]:
tfidfDicts = [{}, {}, {}]
for i, data in enumerate([titles, keywords, descriptions]):
    bank = TfIdfBank(tfIdfVect.transform(data))
    for numb, query in queriesDict.items():
        numbsDocs = queriesDocsDict[numb]
        words = query.strip().split()
        tfidfs = []
        for numbDoc in numbsDocs:
            if bank._indexPages[numbDoc] == -1:
                tfidfs.append(None)
            else:
                tfidfs.append(sum(list(map(lambda x: bank.tfidf(x, numbDoc), words))))
        tfidfDicts[i][numb] = tfidfs        

In [15]:
titlePath = '../data/features/tfidfTitle'
keywordsPath = '../data/features/tfidfKeywords'
descPath = '../data/features/tfidfDesc'

json.dump(tfidfDicts[0], open(titlePath, 'w'))
json.dump(tfidfDicts[1], open(keywordsPath, 'w'))
json.dump(tfidfDicts[2], open(descPath, 'w'))

In [ ]:
data = {"numbs":[], "strings":[]}
sizeMemory = 0
for numb in procNumbs:
    filename = templateJson.format(numb)
    path = '{:s}/{:s}'.format(textdataPath, filename)
    pageDict = json.load(open(path))
    text = []
    for value in pageDict['text'].values():
        text.extend(value)
    text = '\n'.join(text)     
    data["numbs"].append(numb)
    data["strings"].append(text)
    sizeMemory += len(text)
    if sizeMemory > 2 * 10 ** 9:
        break    

In [ ]:
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import TweetTokenizer

rusStopwords = stopwords.words('russian')
tokenizer = TweetTokenizer().tokenize
tfIdfVect = TfidfVectorizer(tokenizer=tokenizer, stop_words=rusStopwords, max_features=50000000)
tfIdfVect.fit(data["strings"])

In [ ]:
tfIdfVectorizerPath = '../data/models/tfIdfFullText.pkl'
joblib.dump(tfIdfVect, tfIdfVectorizerPath)

In [1]:
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
stemmer  = SnowballStemmer('russian')
for word in stopwords.words('russian'):
    print(word, stemmer.stem(word))

и и
в в
во во
не не
что что
он он
на на
я я
с с
со со
как как
а а
то то
все все
она он
так так
его ег
но но
да да
ты ты
к к
у у
же же
вы вы
за за
бы бы
по по
только тольк
ее е
мне мне
было был
вот вот
от от
меня мен
еще ещ
нет нет
о о
из из
ему ем
теперь тепер
когда когд
даже даж
ну ну
вдруг вдруг
ли ли
если есл
уже уж
или ил
ни ни
быть быт
был был
него нег
до до
вас вас
нибудь нибуд
опять опя
уж уж
вам вам
ведь вед
там там
потом пот
себя себ
ничего нич
ей е
может может
они он
тут тут
где где
есть ест
надо над
ней не
для для
мы мы
тебя теб
их их
чем чем
была был
сам сам
чтоб чтоб
без без
будто будт
чего чег
раз раз
тоже тож
себе себ
под под
будет будет
ж ж
тогда тогд
кто кто
этот этот
того тог
потому пот
этого эт
какой как
совсем совс
ним ним
здесь зде
этом эт
один один
почти почт
мой мо
тем тем
чтобы чтоб
нее не
сейчас сейчас
были был
куда куд
зачем зач
всех всех
никогда никогд
можно можн
при при
наконец наконец
два два
об об
другой друг
хоть хот
после посл
над над
больше больш
тот то

In [ ]:
import json
tfIdfVectorizerPath = '../data/models/randomNotNorm.pkl' 
joblib.load(tfIdfVectorizerPath)